# Lecture 10: Basic CNN

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch import cuda

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10) # 320 -> 10

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1) # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)

In [3]:
# MNIST Dataset
train_dataset = datasets.MNIST(root="./data/",
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root="./data/",
                              train=False,
                              transform=transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
batch_size = 32
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')

Training MNIST Model on cuda


In [5]:
# Data Loader (Input Pipeline)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=False)

In [6]:
model = Net()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [7]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx%100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({(100. * batch_idx / len(train_loader)):.6f})%]\tLoss: {loss.data.item():.6f}")

In [8]:
def test():
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # sum up batch loss
            test_loss += criterion(output, target).item()

            # get the index of the max log-probability
            pred = torch.max(output.data, 1)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f"Test set : Average loss = {test_loss:.4f}, Accuracy = {correct}/{len(test_loader.dataset)} ({(100. * correct / len(test_loader.dataset)):.0f}%)\n\n")

In [9]:
epochs = 10
for epoch in range(1, epochs+1):
    train(epoch)
    test()

<ipython-input-2-9cb157d51835>:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0.000000)%]	Loss: 2.294043
Train Epoch: 1 [3200/60000 (5.333333)%]	Loss: 1.394845
Train Epoch: 1 [6400/60000 (10.666667)%]	Loss: 0.671720
Train Epoch: 1 [9600/60000 (16.000000)%]	Loss: 0.330910
Train Epoch: 1 [12800/60000 (21.333333)%]	Loss: 0.150717
Train Epoch: 1 [16000/60000 (26.666667)%]	Loss: 0.158077
Train Epoch: 1 [19200/60000 (32.000000)%]	Loss: 0.095075
Train Epoch: 1 [22400/60000 (37.333333)%]	Loss: 0.197645
Train Epoch: 1 [25600/60000 (42.666667)%]	Loss: 0.284837
Train Epoch: 1 [28800/60000 (48.000000)%]	Loss: 0.164288
Train Epoch: 1 [32000/60000 (53.333333)%]	Loss: 0.206897
Train Epoch: 1 [35200/60000 (58.666667)%]	Loss: 0.197159
Train Epoch: 1 [38400/60000 (64.000000)%]	Loss: 0.158325
Train Epoch: 1 [41600/60000 (69.333333)%]	Loss: 0.089347
Train Epoch: 1 [44800/60000 (74.666667)%]	Loss: 0.060877
Train Epoch: 1 [48000/60000 (80.000000)%]	Loss: 0.100108
Train Epoch: 1 [51200/60000 (85.333333)%]	Loss: 0.349708
Train Epoch: 1 [54400/60000 (90.666667)%